In [1]:
import rasterio
import numpy as np
import os
# Directory where year-to-year LST change rasters are stored
change_dir = '../uhi-analysis/change-detection-results'
lst_change_files = sorted([f for f in os.listdir(change_dir) if f.startswith('LST_change_') and f.endswith('.tif')])

cumulative_change = None

for idx, file in enumerate(lst_change_files):
    file_path = os.path.join(change_dir, file)
    with rasterio.open(file_path) as src:
        data = src.read(1)
        data = np.where(data == src.nodata, 0, data)

        if cumulative_change is None:
            cumulative_change = data
            meta = src.meta.copy()
        else:
            cumulative_change += data

# Save cumulative raster
output_path = os.path.join(change_dir, 'LST_cumulative_change_2013_2024.tif')
with rasterio.open(output_path, 'w', **meta) as dst:
    dst.write(cumulative_change, 1)

print(f"Cumulative LST Change Raster Saved: {output_path}")


NameError: name 'meta' is not defined

In [ ]:

import matplotlib.pyplot as plt
import matplotlib.colors as colors

def compute_and_plot_cumulative_change(data_dir, save_results=False, output_dir=None, contrast_level=10):
    """
    Compute and plot cumulative percent change for NDVI and LST from 2013 to 2024.

    Args:
        data_dir (str): Directory with NDVI_LST_{year}.tif files.
        save_results (bool): If True, saves the cumulative change rasters.
        output_dir (str): Directory to save outputs if save_results=True.
        contrast_level (int): Controls color divergence for LST map. Higher = more drastic.
    """
    years = list(range(2013, 2025))
    ndvi_changes = []
    lst_changes = []

    for i in range(len(years) - 1):
        y1, y2 = years[i], years[i+1]
        file1 = os.path.join(data_dir, f"NDVI_LST_{y1}.tif")
        file2 = os.path.join(data_dir, f"NDVI_LST_{y2}.tif")

        with rasterio.open(file1) as src1, rasterio.open(file2) as src2:
            ndvi1 = src1.read(1).astype(np.float32)
            lst1 = src1.read(2).astype(np.float32)
            ndvi2 = src2.read(1).astype(np.float32)
            lst2 = src2.read(2).astype(np.float32)
            profile = src1.profile

        epsilon = 1e-6
        ndvi_pct = ((ndvi2 - ndvi1) / (ndvi1 + epsilon)) * 100
        lst_pct = ((lst2 - lst1) / (lst1 + epsilon)) * 100

        ndvi_changes.append(ndvi_pct)
        lst_changes.append(lst_pct)

        print(f"Processed: {y1} → {y2}")

    # Compute average percent change
    ndvi_cumulative = np.mean(ndvi_changes, axis=0)
    lst_cumulative = np.mean(lst_changes, axis=0)

    # 📊 Map 1: NDVI Change
    plt.figure(figsize=(8, 6))
    ndvi_map = plt.imshow(ndvi_cumulative, cmap='RdYlGn', vmin=-50, vmax=50)
    plt.title("🌱 Average NDVI % Change (2013–2024)", fontsize=14)
    plt.colorbar(ndvi_map, label="% Change", shrink=0.8)
    plt.axis('off')
    plt.show()

    # 📊 Map 2: LST Change with Enhanced Contrast
    plt.figure(figsize=(8, 6))
    lst_map = plt.imshow(lst_cumulative, cmap='coolwarm',
                         norm=colors.TwoSlopeNorm(vmin=-contrast_level, vcenter=0, vmax=contrast_level))
    plt.title(f"🔥 LST % Change (Enhanced Contrast ±{contrast_level}%)", fontsize=14)
    plt.colorbar(lst_map, label="% Change (LST)", shrink=0.8)
    plt.axis('off')
    plt.show()

    # Save rasters (optional)
    if save_results and output_dir:
        os.makedirs(output_dir, exist_ok=True)
        profile.update(dtype=rasterio.float32, count=1)

        ndvi_out = os.path.join(output_dir, 'NDVI_CumulativeChange_2013_2024.tif')
        with rasterio.open(ndvi_out, 'w', **profile) as dst:
            dst.write(ndvi_cumulative.astype(np.float32), 1)
        print(f"Saved NDVI cumulative change to: {ndvi_out}")

        lst_out = os.path.join(output_dir, 'LST_CumulativeChange_2013_2024.tif')
        with rasterio.open(lst_out, 'w', **profile) as dst:
            dst.write(lst_cumulative.astype(np.float32), 1)
        print(f"Saved LST cumulative change to: {lst_out}")

# Example Usage
compute_and_plot_cumulative_change(
    data_dir="../uhi-analysis/gee_exports",
    save_results=True,
    output_dir="../uhi-analysis/change-detection-cumulative",
    contrast_level=1  
)


In [ ]:

from mpl_toolkits.mplot3d import Axes3D

def plot_3d_surface(raster_path, title, cmap='viridis', zlim=None):
    with rasterio.open(raster_path) as src:
        data = src.read(1)
        data = np.nan_to_num(data, nan=0.0)  # Replace NaN for 3D plot
        transform = src.transform

    rows, cols = data.shape
    x = np.linspace(0, cols, cols)
    y = np.linspace(0, rows, rows)
    X, Y = np.meshgrid(x, y)

    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')
    surf = ax.plot_surface(X, Y, data, cmap=cmap, linewidth=0, antialiased=False)

    ax.set_title(title, fontsize=14)
    fig.colorbar(surf, shrink=0.6, aspect=10)

    if zlim:
        ax.set_zlim(zlim)

    plt.show()

# Example usage
ndvi_path = "../uhi-analysis/change-detection-cumulative/NDVI_CumulativeChange_2013_2024.tif"
lst_path = "../uhi-analysis/change-detection-cumulative/LST_CumulativeChange_2013_2024.tif"

plot_3d_surface(ndvi_path, title="3D NDVI Cumulative Change", cmap='YlGn', zlim=(-50, 50))
plot_3d_surface(lst_path, title="3D LST Cumulative Change", cmap='coolwarm', zlim=(-10, 10))


In [ ]:
import plotly.graph_objects as go

def interactive_3d_plot(raster_path, title, colorscale='Viridis', zmin=None, zmax=None):
    with rasterio.open(raster_path) as src:
        data = src.read(1)
        data = np.nan_to_num(data, nan=0.0)

    fig = go.Figure(data=[go.Surface(z=data, colorscale=colorscale, cmin=zmin, cmax=zmax)])
    fig.update_layout(title=title, autosize=True,
                      width=800, height=700,
                      margin=dict(l=50, r=50, b=65, t=90))

    fig.show()

# Example usage
interactive_3d_plot(ndvi_path, "Interactive 3D NDVI Change", colorscale='Greens', zmin=-50, zmax=50)
interactive_3d_plot(lst_path, "Interactive 3D LST Change", colorscale='RdBu', zmin=-10, zmax=10)
